In [1]:
from transformers import AutoTokenizer, RobertaForSequenceClassification, pipeline
from collections import Counter
from tqdm import tqdm
import pandas as pd
import torch
import re

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # Use GPU if available

1. Sentiment Analysis (Positive, Negative, Neutral) :

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
model = RobertaForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

In [ ]:
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model=model,
    tokenizer=tokenizer,
    device=device,
    max_length=512,
    truncation=True
)

In [3]:
data = pd.read_csv('../data/results/data.csv')

# drop nan in body
data = data.dropna(subset=['body'])

# cleaning
data['body2'] = data['body'].str.replace('\n', ' ')
data['body2'] = data['body2'].str.strip()
data['body2'] = data['body2'].apply(lambda x: re.sub(' +', ' ', x))
data['body2'] = data['body2'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
data['body2'] = data['body2'].apply(lambda x: re.sub(r'http\S+', '', x))

In [ ]:
tqdm.pandas()
data['sentiment'] = data['body2'].progress_apply(sentiment_pipeline)

In [ ]:
data['sentiment_score'] = data['sentiment'].apply(lambda x:x[0]['score'])
data['sentiment'] = data['sentiment'].apply(lambda x:x[0]['label'])

In [ ]:
data.to_csv('../data/results/data_sentiment.csv')

In [17]:
Counter(list(data['sentiment']))

Counter({'neutral': 647612, 'negative': 429790, 'positive': 215097})

2. Emotion Analysis (Anger, Fear, Joy, Sadness, Surprise, Disgust, Neutral) :

In [4]:
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=False, device=device, truncation=True, verbose=True)

/home/chel/folder/RedDash/env/lib/python3.11/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [5]:
tqdm.pandas()
# _lis = data[data['Topic']==2].body2.progress_apply(classifier)
data['emotion'] = data.body2.progress_apply(classifier)

100%|██████████| 1276424/1276424 [4:27:11<00:00, 79.62it/s]  


In [6]:
data['emotion_score'] = data['emotion'].apply(lambda x:x[0]['score'])
data['emotion'] = data['emotion'].apply(lambda x:x[0]['label'])

In [13]:
Counter(list(data['emotion']))

Counter({'neutral': 661310,
         'joy': 134459,
         'anger': 121800,
         'surprise': 107678,
         'sadness': 101110,
         'fear': 96582,
         'disgust': 53485})

In [14]:
data.to_csv('../data/results/data_emotions.csv')